In [1]:
import sys
import random
import os
import fitz 
import cv2
import pyautogui
import numpy as np
import time
from PIL import ImageGrab
from datetime import datetime
import threading
from PyQt5.QtCore import Qt, QUrl, QSize, QTimer
from PyQt5.QtGui import QImage, QPixmap, QColor, QPainter, QBrush
from PyQt5.QtWidgets import QApplication, QWidget, QVBoxLayout, QHBoxLayout, QLabel, QScrollArea, QPushButton, QMainWindow, QSlider, QGridLayout, QLineEdit, QFrame, QSizePolicy, QMessageBox, QSpacerItem, QRadioButton
from PyQt5.QtWebEngineWidgets import QWebEngineView
from PyQt5.QtMultimedia import QMediaPlayer, QMediaContent
from PyQt5.QtMultimediaWidgets import QVideoWidget

class RecordingManager:
    def __init__(self):
        self.stop_event = threading.Event()
        self.screen_thread = None
        self.webcam_thread = None
        self.is_recording = False
  
    def get_timestamped_filename(self, prefix, extension):
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        return f"{prefix}_{timestamp}.{extension}"

    def record_screen(self):
        output_file = self.get_timestamped_filename("screen_recording", "mp4")
        screen_size = pyautogui.size()
        fourcc = cv2.VideoWriter_fourcc(*"mp4v")
        fps = 20.0  # Set the same frame rate as the webcam recording
        out = cv2.VideoWriter(output_file, fourcc, fps, (screen_size.width, screen_size.height))

        frame_time = 1 / fps
        last_capture_time = time.time()

        while not self.stop_event.is_set():
            current_time = time.time()
            if current_time - last_capture_time >= frame_time:
                img = ImageGrab.grab(bbox=(0, 0, screen_size.width, screen_size.height))
                frame = np.array(img)
                frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
                out.write(frame)
                last_capture_time = current_time

            else:
                time.sleep(0.001)

        out.release()
        print(f"Screen recording saved to {output_file}")
    
    def record_webcam(self):
        output_file = self.get_timestamped_filename("webcam_recording", "mp4")
        cap = cv2.VideoCapture(1) 
        if not cap.isOpened():
            print("Error: Could not open webcam.")
            return

        cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_file, fourcc, 20.0, (640, 480))

        while not self.stop_event.is_set():
            ret, frame = cap.read()
            if not ret:
                print("Failed to capture frame from webcam")
                break
            out.write(frame)
            time.sleep(0.05)

        cap.release()
        out.release()
        cv2.destroyAllWindows()
        print(f"Webcam recording saved to {output_file}")

    def start_recordings(self):
        if not self.is_recording:
            self.is_recording = True
            self.stop_event.clear()
            self.screen_thread = threading.Thread(target=self.record_screen)
            self.webcam_thread = threading.Thread(target=self.record_webcam)
            self.screen_thread.start()
            self.webcam_thread.start()
            time.sleep(0.1)

    def stop_recordings(self):
        if self.is_recording:
            self.is_recording = False
            self.stop_event.set()
            time.sleep(0.1)
            if self.screen_thread:
                self.screen_thread.join()
            if self.webcam_thread:
                self.webcam_thread.join()
   
class InitialWindow(QMainWindow):
    def __init__(self):
        super().__init__()
        self.recording_manager = RecordingManager()
        self.setWindowTitle("Main Application")
        self.showMaximized()  # Maximize the window to full screen

        layout = QVBoxLayout()
        start_button = QPushButton("Start Session")
        start_button.setFixedSize(200, 40)
        start_button.clicked.connect(self.start_session)

        button_layout = QHBoxLayout()
        button_layout.addStretch()
        button_layout.addWidget(start_button)
        button_layout.addStretch()

        layout.addLayout(button_layout)

        container = QWidget()
        container.setLayout(layout)
        self.setCentralWidget(container)

    def start_session(self):
        self.recording_manager.start_recordings()
        self.pdf_task_window = PDFTaskWindow(self.recording_manager)
        self.pdf_task_window.show()
        self.close()

    # def closeEvent(self, event):
    #     """Handle application close event to stop recordings."""
    #     if self.recording_manager.is_recording:
    #         self.recording_manager.stop_recordings()
    #     event.accept()

class PDFTaskWindow(QWidget):
    def __init__(self, recording_manager):
        super().__init__()
        self.recording_manager = recording_manager
        self.setWindowTitle("PDF Task")
        self.showMaximized()

        # Initialize layout
        self.layout = QVBoxLayout()

        # Initialize QScrollArea
        self.scroll_area = QScrollArea()
        self.scroll_area.setSizePolicy(QSizePolicy.Expanding, QSizePolicy.Expanding)
        self.scroll_area.setWidgetResizable(True)

        # Create a widget to hold the PDF pages
        self.scroll_widget = QWidget()
        self.scroll_layout = QVBoxLayout()

        self.scroll_widget.setLayout(self.scroll_layout)
        self.scroll_area.setWidget(self.scroll_widget) 

        self.layout.addWidget(self.scroll_area)

        # Add button at the bottom of the scrollable area
        self.next_button = QPushButton("Next", self)
        self.next_button.setFixedSize(200, 40)
        self.next_button.clicked.connect(self.open_form1)
        self.layout.addWidget(self.next_button)

        self.setLayout(self.layout)
        self.load_pdf()

    def load_pdf(self):
        pdf_path = "Minor Synopsis.pdf"  # Adjust the path as needed
        doc = fitz.open(pdf_path)

        # Make sure scroll_area is initialized before accessing it
        available_width = self.scroll_area.viewport().width()  # Get the current scroll area width

        # Clear existing widgets
        for i in reversed(range(self.scroll_layout.count())):
            widget = self.scroll_layout.itemAt(i).widget()
            if widget:
                widget.deleteLater()

        for page_num in range(len(doc)):
            page = doc.load_page(page_num)
            pix = page.get_pixmap()
            img = QImage(pix.samples, pix.width, pix.height, pix.stride, QImage.Format_RGB888)
            pixmap = QPixmap.fromImage(img)

            # Scale the pixmap to fit the width of the scroll area while maintaining aspect ratio
            scaled_pixmap = pixmap.scaled(int(available_width), int(available_width * pixmap.height() / pixmap.width()), Qt.KeepAspectRatio, Qt.SmoothTransformation)

            pdf_label = QLabel()
            pdf_label.setPixmap(scaled_pixmap)
            pdf_label.setAlignment(Qt.AlignCenter)  # Center the image horizontally
            self.scroll_layout.addWidget(pdf_label)

        doc.close()

    def open_form1(self):
        self.form1_window = Form1Window(self.recording_manager)
        self.form1_window.show()
        self.close()
  
class Form1Window(QWidget):
    def __init__(self, recording_manager):
        super().__init__()
        self.recording_manager = recording_manager
        self.setWindowTitle("Form 1")
        #self.setGeometry(100, 100, 800, 600)
        self.showMaximized()

        layout = QVBoxLayout()
        instruction_label = QLabel("Please fill out the Google Form below.")
        layout.addWidget(instruction_label)
        
        spacer_item = QSpacerItem(20, 10, QSizePolicy.Expanding, QSizePolicy.Minimum)  # Adjust width and height as needed
        layout.addItem(spacer_item)

        self.web_view = QWebEngineView()
        self.web_view.setUrl(QUrl("https://forms.gle/Hi2qHzYBgpBg8Wyx5"))  # Replace with your Form 2 URL
        
        # Add Prev and Next buttons
        self.button_layout = QHBoxLayout()
        self.button_layout.setAlignment(Qt.AlignLeft)

        self.prev_button = QPushButton("Prev")
        self.prev_button.setFixedSize(200, 40)
        self.prev_button.clicked.connect(self.open_pdf_task)
       

        self.next_button = QPushButton("Next")
        self.next_button.setFixedSize(200, 40)
        self.next_button.clicked.connect(self.open_video_task)

        # Add both buttons without extra spacing
        self.button_layout.addWidget(self.prev_button)
        #self.button_layout.addSpacing(10)  # Add a small gap between buttons
        self.button_layout.addWidget(self.next_button)


        layout.addWidget(self.web_view)
        layout.addLayout(self.button_layout)
        self.web_view.setFixedHeight(650) 
        
        self.setLayout(layout)

    def open_pdf_task(self):
        self.pdf_task_window = PDFTaskWindow(self.recording_manager)
        self.pdf_task_window.show()
        self.close()

    def open_video_task(self):
        self.video_task_window = VideoTaskWindow(self.recording_manager)
        self.video_task_window.show()
        self.close()

class VideoTaskWindow(QWidget):
    def __init__(self, recording_manager):
        super().__init__()
        self.recording_manager = recording_manager
        self.setWindowTitle("Video Task")
        self.showMaximized()
        self.setLayout(QVBoxLayout())

        self.media_player = QMediaPlayer()
        video_widget = QVideoWidget()
        self.media_player.setVideoOutput(video_widget)
        self.layout().addWidget(video_widget)

        # Use an absolute path for the video file
        video_path = "video_genre.mp4"  # Adjust the path as needed
        absolute_path = os.path.abspath(video_path)
        if not os.path.exists(absolute_path):
            print(f"Error: Video file not found at {absolute_path}")
        else:
            self.media_player.setMedia(QMediaContent(QUrl.fromLocalFile(absolute_path)))

        # Error handling
        self.media_player.error.connect(self.handle_error)
        self.media_player.stateChanged.connect(self.handle_state_change)

    
        # Add button to proceed to Form 2
        self.next_button = QPushButton("Next")
        self.next_button.setFixedSize(200, 40)
        self.next_button.clicked.connect(self.open_form2)

        self.layout().addWidget(self.next_button, Qt.AlignmentFlag.AlignBottom)

        self.media_player.play()

    def handle_state_change(self, state):
        if state == QMediaPlayer.State.StoppedState:
            self.open_form2()

    def handle_error(self, error):
        QMessageBox.critical(self, "Error", f"Error: {self.media_player.errorString()}")

    def open_form2(self):
        # Stop the video playback
        self.media_player.stop()
        
        # Proceed to Form 2
        self.form2_window = Form2Window(self.recording_manager)
        self.form2_window.show()
        self.close()

class Form2Window(QWidget):
    def __init__(self, recording_manager):
        super().__init__()
        self.recording_manager = recording_manager
        self.setWindowTitle("Form 2")
        #self.setGeometry(100, 100, 800, 600)
        self.showMaximized()

        layout = QVBoxLayout()
        instruction_label = QLabel("Please fill out the Google Form below.")
        layout.addWidget(instruction_label)
        
        spacer_item = QSpacerItem(20, 10, QSizePolicy.Expanding, QSizePolicy.Minimum)  # Adjust width and height as needed
        layout.addItem(spacer_item)

        self.web_view = QWebEngineView()
        self.web_view.setUrl(QUrl("https://forms.gle/Hi2qHzYBgpBg8Wyx5"))  # Replace with your Form 2 URL
        
        self.next_button = QPushButton("Next")
        self.next_button.setFixedSize(200, 40)
        self.next_button.clicked.connect(self.open_sudoku)
        
        layout.addWidget(self.web_view)
        layout.addWidget(self.next_button)
        self.web_view.setFixedHeight(650) 
        
        self.setLayout(layout)

    def open_sudoku(self):
        self.sudoku_window = SudokuWindow(self.recording_manager)
        self.sudoku_window.show()
        self.close()
    
# Function to extract an alternate 3x3 grid
def extract_alternate_grid(matrix, start_row, start_col):
    grid = []
    for i in range(start_row, start_row + 3):
        row = []
        for j in range(start_col, start_col + 3):
            row.append(matrix[i][j])
        grid.append(row)
    return grid

class SudokuWindow(QWidget):
    def __init__(self, recording_manager):
        super().__init__()
        self.recording_manager = recording_manager
        self.setWindowTitle("Sudoku Puzzle")
        self.showMaximized()

        # Main layout for the window
        main_layout = QVBoxLayout(self)
    
        # Create outer container
        self.outer_container = QWidget()
        outer_layout = QVBoxLayout(self.outer_container)
        outer_layout.setContentsMargins(0, 0, 0, 0)  # Remove margins from the outer container layout
        outer_layout.setAlignment(Qt.AlignCenter)

        # Create inner container for the grid
        self.inner_container = QWidget()
        self.grid_layout = QGridLayout(self.inner_container)
        self.grid_layout.setSpacing(0)  # Set spacing to zero to remove gaps between cells
        self.grid_layout.setContentsMargins(0, 0, 0, 0)  # Remove margins from the grid layout

        # Set fixed size for the inner container
        self.inner_container.setFixedSize(400, 400)

        initial_grid = [
            [7, 4, 0, 0, 3, 0, 0, 1, 0],
            [0, 1, 9, 0, 6, 8, 5, 0, 2],
            [0, 0, 3, 0, 0, 4, 3, 0, 0],
            [0, 5, 6, 3, 7, 0, 0, 0, 1],
            [0, 0, 1, 8, 0, 0, 0, 9, 5],
            [0, 9, 0, 0, 2, 0, 6, 0, 0],
            [1, 0, 3, 4, 0, 7, 2, 0, 0],
            [5, 0, 0, 2, 0, 0, 0, 0, 9],
            [0, 8, 0, 0, 0, 1, 4, 7, 0]
        ]

        cell_size = 40  # Size for better readability

        # Determine alternate blocks using the extract_alternate_grid function
        alternate_blocks = [(0, 3), (0, 4), (0, 5),
                            (1, 3), (1, 4), (1, 5),
                            (2, 3), (2, 4), (2, 5),
                            (3, 0), (3, 1), (3, 2),
                            (4, 0), (4, 1), (4, 2),
                            (5, 0), (5, 1), (5, 2),
                            (3, 6), (3, 7), (3, 8),
                            (4, 6), (4, 7), (4, 8),
                            (5, 6), (5, 7), (5, 8),
                            (6, 3), (6, 4), (6, 5),
                            (7, 3), (7, 4), (7, 5),
                            (8, 3), (8, 4), (8, 5), 
                            ]

        for i in range(9):
            for j in range(9):
                cell = QLineEdit()
                cell.setMaxLength(1)
                cell.setFixedSize(cell_size, cell_size)
                cell.setAlignment(Qt.AlignCenter)

                if (i, j) in alternate_blocks:
                    base_style = "QLineEdit { background-color: #dfeffc; color: #000000; font-size: 14pt; padding: 0px; margin: 0px; border: 1px solid #AAAAAA; }"
                else:
                    base_style = "QLineEdit { background-color: #FFFFFF; color: #000000; font-size: 14pt; padding: 0px; margin: 0px; border: 1px solid #AAAAAA; }"

                if initial_grid[i][j] != 0:
                    cell.setText(str(initial_grid[i][j]))
                    cell.setReadOnly(True)

                cell.setStyleSheet(base_style)
                self.grid_layout.addWidget(cell, i, j)

        # Add inner container to the outer container
        outer_layout.addWidget(self.inner_container)

        # Add outer container to the main layout
        main_layout.addWidget(self.outer_container)

        # Add Submit button
        self.submit_button = QPushButton("Submit and Next")
        self.submit_button.setFixedSize(200, 40)
        self.submit_button.clicked.connect(self.go_to_memory_test)
        main_layout.addWidget(self.submit_button)

    def go_to_memory_test(self):
        try:
            print("Sudoku puzzle submitted!")
            self.memory_test_window = MemoryTestWindow(self.recording_manager)
            self.memory_test_window.show()
            self.close()
        except Exception as e:
            print(f"Error transitioning to Memory Test: {e}")
            QMessageBox.critical(self, "Error", f"Failed to load Memory Test: {str(e)}")

class MemoryTestWindow(QWidget):
    def __init__(self, recording_manager):
        super().__init__()
        self.recording_manager = recording_manager
        self.setWindowTitle("Memory Test")
        self.showMaximized()

        self.circle_widget = CircleWidget()
        self.init_ui()

    def init_ui(self):
        layout = QVBoxLayout(self)
        
        instruction_label = QLabel(
            'A memory test will be shown where five circles will be colored one at a time.\n'
            'After they are shown, click all those circles which were colored earlier when shown.',
            self
        )
        instruction_label.setAlignment(Qt.AlignTop | Qt.AlignLeft)  
        instruction_label.setStyleSheet("padding:10px; font-size: 16px;")
        layout.addWidget(instruction_label)

        # Center the CircleWidget
        circle_layout = QHBoxLayout()
        circle_layout.addStretch()
        circle_layout.addWidget(self.circle_widget)
        circle_layout.addStretch()
        layout.addLayout(circle_layout)

        # Next button at bottom left
        next_button = QPushButton('Next', self)
        next_button.setFixedSize(200, 40)
        next_button.clicked.connect(self.go_to_logical_reasoning)
        
        button_layout = QHBoxLayout()
        button_layout.addWidget(next_button)
        button_layout.addStretch()
        layout.addLayout(button_layout)

        # Set up the sequence for highlighting circles
        self.circle_widget.sequence = random.sample(range(len(self.circle_widget.circles)), 5)
        self.timer = QTimer(self)
        self.timer.timeout.connect(self.circle_widget.highlight_next_circle)
        self.timer.setInterval(1000)
        QTimer.singleShot(500, self.timer.start)  # Slight delay before starting

        # Stop the sequence after highlights and reset circles
        QTimer.singleShot(6000, self.timer.stop)
        QTimer.singleShot(6000, self.circle_widget.reset_circles)
        QTimer.singleShot(6000, self.circle_widget.start_memory_test)

    def go_to_logical_reasoning(self):
        print("Memory test submitted!")
        self.logical_reasoning_window = LogicalReasoningWindow(self.recording_manager)
        self.logical_reasoning_window.show()
        self.close()

class CircleWidget(QWidget):
    def __init__(self):
        super().__init__()
        self.setFixedSize(550, 550)
        
        self.positions = [
            (75, 75),    (175, 100),  (300, 175),  (450, 90),
            (125, 250),  (275, 225),  (425, 300),  (500, 200),
            (150, 450),  (375, 400)
        ]
        self.circles = self.generate_circles()
        self.sequence = []
        self.current_index = -1
        self.memory_test_started = False
        self.correct_clicks = []
        self.max_clicks = 5

    def generate_circles(self):
        return [{"position": pos, "radius": 18, "color": QColor('white')} for pos in self.positions]

    def paintEvent(self, event):
        painter = QPainter(self)
        painter.setRenderHint(QPainter.Antialiasing)
        painter.setBrush(QBrush(QColor('skyblue')))
        painter.drawRect(self.rect())  # Background color only for CircleWidget

        for circle in self.circles:
            painter.setBrush(QBrush(circle["color"]))
            x, y = circle["position"]
            painter.drawEllipse(x - circle["radius"], y - circle["radius"],
                                circle["radius"] * 2, circle["radius"] * 2)

    def highlight_next_circle(self):
        if self.current_index >= 0:
            self.circles[self.sequence[self.current_index]]["color"] = QColor('white')
        
        self.current_index += 1

        if self.current_index < len(self.sequence):
            self.circles[self.sequence[self.current_index]]["color"] = QColor('red')
        else:
            self.reset_circles()
            self.memory_test_started = True
            self.current_index = -1

        self.update()

    def reset_circles(self):
        for circle in self.circles:
            circle["color"] = QColor('white')
        self.update()

    def start_memory_test(self):
        self.memory_test_started = True
        self.update()

    def mousePressEvent(self, event):
        if not self.memory_test_started:
            return

        x, y = event.x(), event.y()
        for i, circle in enumerate(self.circles):
            cx, cy = circle["position"]
            if (x - cx) ** 2 + (y - cy) ** 2 <= circle["radius"] ** 2:
                if circle["color"] == QColor('white'):
                    if len(self.correct_clicks) < self.max_clicks:
                        circle["color"] = QColor('green')
                        self.correct_clicks.append(i)
                else:
                    circle["color"] = QColor('white')
                    if i in self.correct_clicks:
                        self.correct_clicks.remove(i)
                self.update()
                break

class LogicalReasoningWindow(QWidget):
    def __init__(self, recording_manager):
        super().__init__()
        self.recording_manager = recording_manager
        self.setWindowTitle("Logical Reasoning")
        self.showMaximized() 
        #self.setGeometry(100, 100, 800, 600)
    
        layout = QVBoxLayout()

        instruction_label = QLabel("Complete the logical reasoning task.")
        instruction_label.setAlignment(Qt.AlignTop | Qt.AlignLeft)  # Align to top-left
        instruction_label.setStyleSheet("padding:5px; font-size: 15px;")  # Add a small padding for visual comfort
        layout.addWidget(instruction_label)

        # Spacer item to control space between instruction label and problem statement
        spacer = QSpacerItem(20, 10, QSizePolicy.Minimum, QSizePolicy.Expanding)
        layout.addItem(spacer)

        # Problem statement using QLabel with plain text
        problem_statement1 = QLabel(
        "Question 1\n\n"
        "In a town, there are three types of people: Philosophers, Scientists, and Mathematicians. The following statements are known:\n\n"
        "All Philosophers are Scientists.\n"
        "Some Scientists are Mathematicians.\n\n"
        "Based on the information provided, which of the following statements must be true?\n"
        "A. All Philosophers are Mathematicians.\n"
        "B. Some Mathematicians are Scientists.\n"
        "C. No Mathematicians are Philosophers.\n"
        "D. All Scientists are Philosophers.\n\n"
        )
        problem_statement1.setAlignment(Qt.AlignTop | Qt.AlignLeft)  # Align to top-left
        problem_statement1.setWordWrap(True)  # Enable word wrapping
        problem_statement1.setStyleSheet("padding:5px; font-size: 15px;")  # Add padding for visual comfort
        layout.addWidget(problem_statement1)
        
        self.answer_input = QLineEdit()
        self.answer_input.setPlaceholderText("Enter your answer here")
        self.answer_input.setMinimumHeight(40)
        layout.addWidget(self.answer_input)


        # Problem Statement 2
        problem_statement2 = QLabel(
            "\n\n\n Question 2\n\n"
            "There are six persons A, B, C, D, E, and F. C is the sister of F. B is the brother of E's husband. D is the father of A and grandfather of F.\n"
            "There are two fathers, three brothers, and a mother in the group. Who is the mother?\n\n"
        )
        problem_statement2.setAlignment(Qt.AlignTop | Qt.AlignLeft)
        problem_statement2.setWordWrap(True)
        problem_statement2.setStyleSheet("padding:5px; font-size: 15px;")
        layout.addWidget(problem_statement2)
        
        # MCQ Options for Problem 2
        mcq_options = QVBoxLayout()
        
        self.option_a = QRadioButton("A. A")
        self.option_b = QRadioButton("B. B")
        self.option_c = QRadioButton("C. C")
        self.option_d = QRadioButton("D. E")

        # Apply custom font size to each option
        self.option_a.setStyleSheet("font-size: 15px;")
        self.option_b.setStyleSheet("font-size: 15px;")
        self.option_c.setStyleSheet("font-size: 15px;")
        self.option_d.setStyleSheet("font-size: 15px;")
        
        mcq_options.addWidget(self.option_a)
        mcq_options.addWidget(self.option_b)
        mcq_options.addWidget(self.option_c)
        mcq_options.addWidget(self.option_d)
        
        layout.addLayout(mcq_options)
        self.next_button = QPushButton("Next")
        self.next_button.setFixedSize(200, 40)
        #self.next_button.clicked.connect(self.open_image_task)
        self.next_button.clicked.connect(self.end_session)
        
        layout.addWidget(self.next_button)
        
        self.setLayout(layout)
    
    # def open_image_task(self):
    #     # Create and show the ImageTaskWindow
    #     self.image_task_window = ImageTaskWindow(self.recording_manager)
    #     self.image_task_window.show()
    #     self.close()

    def end_session(self):
        # self.recording_manager.stop_recordings()
        self.final_window = FinalMessageWindow(self.recording_manager)
        self.final_window.show()
        self.close()
    
# class ImageTaskWindow(QWidget):
#     def __init__(self, recording_manager):
#     # def __init__(self):
#         super().__init__()
#         self.recording_manager = recording_manager
#         self.setWindowTitle("Image Task")
#         self.showMaximized()

#         layout = QVBoxLayout()
        
#         # Load and display the image
#         image_path = "image_task.jpg"  # Replace with the path to your image
#         self.image_label = QLabel()
#         pixmap = QPixmap(image_path)
#         self.image_label.setPixmap(pixmap.scaled(800, 600, Qt.KeepAspectRatio))
#         self.image_label.setAlignment(Qt.AlignCenter)
#         layout.addWidget(self.image_label)
        
#         # Add the question below the image
#         question_label = QLabel("What number do you see in the image?")
#         question_label.setAlignment(Qt.AlignCenter)
#         layout.addWidget(question_label)
        
#         # Add a text input for the user to enter their answer
#         self.answer_input = QLineEdit()
#         self.answer_input.setPlaceholderText("Enter your answer here")
#         self.answer_input.setMinimumHeight(40)
#         layout.addWidget(self.answer_input)
        
#         # Add a button to proceed to the next task
#         self.next_button = QPushButton("Next")
#         self.next_button.setFixedSize(200, 40)
#         self.next_button.clicked.connect(self.complete_session)
#         layout.addWidget(self.next_button)
        
#         self.setLayout(layout)
        
#     def complete_session(self):
#         self.final_message_window = FinalMessageWindow(self.recording_manager)
#         self.final_message_window.show()
#         self.close()

class FinalMessageWindow(QWidget):
    def __init__(self, recording_manager):
        super().__init__()
        self.recording_manager = recording_manager
        self.setWindowTitle("Completion Message")
        self.showMaximized()  # Maximize the window to full screen

        layout = QVBoxLayout()
        message_label = QLabel("Thanks, you have completed all your tasks!")
        message_label.setAlignment(Qt.AlignCenter)
        layout.addWidget(message_label)

        button_layout = QHBoxLayout()
        self.end_button = QPushButton("End Session")
        self.end_button.setFixedSize(200, 40)
        self.end_button.clicked.connect(self.end_session)
        button_layout.addStretch()
        button_layout.addWidget(self.end_button)
        button_layout.addStretch()

        layout.addLayout(button_layout)
        self.setLayout(layout)

    def end_session(self):
        self.recording_manager.stop_recordings()
        self.initial_window = InitialWindow()
        self.initial_window.show()
        self.close()

if __name__ == "__main__":
    app = QApplication(sys.argv)
    main_window = InitialWindow()
    main_window.show()
    sys.exit(app.exec())


2024-08-26 21:58:55.267 python[1218:9759965] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.
js: Refused to set the document's base URI to 'https://proxy.link.app/' because it violates the following Content Security Policy directive: "base-uri 'self'".

js: Refused to set the document's base URI to 'https://proxy.link.app/' because it violates the following Content Security Policy directive: "base-uri 'self'".



Sudoku puzzle submitted!
Memory test submitted!
Screen recording saved to screen_recording_20240826_215854.mp4
Webcam recording saved to webcam_recording_20240826_215854.mp4


SystemExit: 0

/opt/anaconda3/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
